In [61]:
from PIL import Image
import torchvision.transforms as transforms
import torch
import numpy as np
import math
import sys

### Training model

#### Implementing custom dataset

In [62]:
sys.path.append('./dataset')
from deep_fashion_2 import DeepFashion2Dataset 

In [63]:
data_path = {
    "train": "../data/train",
    "val": "../data/validation",
}

In [64]:
def collate_fn(batch):
    return tuple(zip(*batch))
    
train_dataset = DeepFashion2Dataset(data_path['train'],1000)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True,collate_fn=collate_fn, num_workers=0)
val_dataset = DeepFashion2Dataset(data_path['val'],1000)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=8, shuffle=True,collate_fn=collate_fn, num_workers=0)

Exception ignored in: <function DeepFashion2Dataset.__del__ at 0x000002907315BC40>
Traceback (most recent call last):
  File "b:\code\artificial-intelligence\outfit-ai\repository\dataset\deep_fashion_2.py", line 49, in __del__
    self.txn.commit() 
    ^^^^^^^^
AttributeError: 'DeepFashion2Dataset' object has no attribute 'txn'
Exception ignored in: <function DeepFashion2Dataset.__del__ at 0x000002907315BC40>
Traceback (most recent call last):
  File "b:\code\artificial-intelligence\outfit-ai\repository\dataset\deep_fashion_2.py", line 49, in __del__
    self.txn.commit() 
    ^^^^^^^^
AttributeError: 'DeepFashion2Dataset' object has no attribute 'txn'


#### Setting Model

In [65]:
sys.path.append('./model')
from faster_r_cnn import FasterRCNNResNet101 

In [66]:
model = FasterRCNNResNet101(num_classes=14)

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

FasterRCNNResNet101(
  (model): FasterRCNN(
    (transform): GeneralizedRCNNTransform(
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        Resize(min_size=(800,), max_size=1333, mode='bilinear')
    )
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=1e-05)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=1e-05)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=1e-05)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [67]:
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

#### Looping train

In [68]:
num_epochs = 5

for epoch in range(num_epochs):
    # Training loop
    model.train() 
    running_loss = 0.0
    
    for i, (images, targets) in enumerate(train_dataloader):
        
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        
        optimizer.zero_grad()

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        
        losses.backward()
        optimizer.step()

        running_loss += losses.item()
        
        print(f"Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{len(train_dataloader)}], Loss: {losses.item()}")

    lr_scheduler.step()

    
print("Training complete!")


RuntimeError: a Tensor with 400 elements cannot be converted to Scalar

#### Saving trained model

In [131]:
torch.save(model.state_dict(), "../model/faster_rcnn_resnet101.pth")